In [1]:
import pandas as pd
import numpy as np

In [5]:
# Load your dataset
df = pd.read_csv("data/houseprice_clean.csv")

In [6]:
print("✅ Dataset Loaded:", df.shape)

✅ Dataset Loaded: (2000, 8)


In [7]:
# ============================
# 1) HANDLE MISSING VALUES
# ============================

# Numerical: fill with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical: fill with mode
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

print("✅ Missing values handled")

✅ Missing values handled


In [15]:
print(df.columns.tolist())


['Township', 'Area', 'State', 'Tenure', 'Type', 'Median_Price', 'Median_PSF', 'Transactions']


In [18]:
# Display initial info
print("Original dataset shape:", df.shape)
print("\nOriginal columns:", df.columns.tolist())

# Create a copy for feature engineering
df_fe = df.copy()

# 1. Area_Transaction_Intensity
print("\n1. Creating Area_Transaction_Intensity...")
area_transaction_totals = df_fe.groupby('Area')['Transactions'].sum()
df_fe['Area_Total_Transactions'] = df_fe['Area'].map(area_transaction_totals)
bins = [0, df_fe['Area_Total_Transactions'].quantile(0.33), df_fe['Area_Total_Transactions'].quantile(0.66), float('inf')]
labels = ['Low', 'Medium', 'High']
df_fe['Area_Transaction_Intensity'] = pd.cut(df_fe['Area_Total_Transactions'], bins=bins, labels=labels)
df_fe.drop('Area_Total_Transactions', axis=1, inplace=True)

# 2. State_Price_Tier
print("2. Creating State_Price_Tier...")
state_median_prices = df_fe.groupby('State')['Median_Price'].median().sort_values(ascending=False)
tier_mapping = {}
for i, (state, _) in enumerate(state_median_prices.items()):
    if i < len(state_median_prices) // 3:
        tier_mapping[state] = 'Tier 1: High-Cost State'
    elif i < 2 * len(state_median_prices) // 3:
        tier_mapping[state] = 'Tier 2: Medium-Cost State'
    else:
        tier_mapping[state] = 'Tier 3: Low-Cost State'
df_fe['State_Price_Tier'] = df_fe['State'].map(tier_mapping)

# 3. Township_Uniqueness_Index
print("3. Creating Township_Uniqueness_Index...")
township_stats = df_fe.groupby('Township').agg({
    'Type': 'nunique',
    'Tenure': 'nunique'
}).reset_index()
township_stats['Township_Uniqueness_Index'] = township_stats['Type'] + township_stats['Tenure']
township_mapping = township_stats.set_index('Township')['Township_Uniqueness_Index'].to_dict()
df_fe['Township_Uniqueness_Index'] = df_fe['Township'].map(township_mapping)

# 4. Price_to_PSF_Ratio
print("4. Creating Price_to_PSF_Ratio...")
df_fe['Median_PSF_clean'] = df_fe['Median_PSF'].replace(0, np.nan)
df_fe['Median_PSF_clean'] = df_fe['Median_PSF_clean'].fillna(df_fe['Median_PSF_clean'].median())
df_fe['Price_to_PSF_Ratio'] = df_fe['Median_Price'] / df_fe['Median_PSF_clean']
df_fe.drop('Median_PSF_clean', axis=1, inplace=True)

# 5. Tenure_Type_Interaction
print("5. Creating Tenure_Type_Interaction...")
df_fe['Tenure_Type_Interaction'] = df_fe['Tenure'] + '_' + df_fe['Type']

# 6. Is_Capital_City
print("6. Creating Is_Capital_City...")
capital_areas = ['Klang Valley', 'Penang Island']
capital_states = ['Wilayah Persekutuan Kuala Lumpur', 'Kuala Lumpur']
df_fe['Is_Capital_City'] = (
    (df_fe['Area'].isin(capital_areas)) | 
    (df_fe['State'].isin(capital_states))
).astype(int)

# 7. Transaction_Volume_Flag
print("7. Creating Transaction_Volume_Flag...")
bins_transactions = [0, 10, 50, float('inf')]
labels_transactions = ['Low_Volume', 'Medium_Volume', 'High_Volume']
df_fe['Transaction_Volume_Flag'] = pd.cut(
    df_fe['Transactions'], 
    bins=bins_transactions, 
    labels=labels_transactions
)

# Display the results
print("\n" + "="*50)
print("FEATURE ENGINEERING COMPLETED")
print("="*50)
print(f"Original dataset shape: {df.shape}")
print(f"New dataset shape: {df_fe.shape}")

# SAVE AS CSV FILE
print("\n" + "="*50)
print("SAVING AS CSV FILE")
print("="*50)

# Save the feature-engineered dataset as CSV
df_fe.to_csv('housepricefeature.csv', index=False)
print("Feature-engineered dataset saved as: malaysia_house_prices_feature_engineered.csv")

# Display confirmation
print(f"File saved successfully!")
print(f"Rows: {df_fe.shape[0]}, Columns: {df_fe.shape[1]}")
print(f"Location: malaysia_house_prices_feature_engineered.csv")

# Show all columns in the saved CSV
print("\nAll columns in the saved CSV file:")
for i, col in enumerate(df_fe.columns.tolist(), 1):
    print(f"{i:2d}. {col}")

# Show first few rows with new features
print("\nFirst 5 rows with new features:")
new_features = [
    'Area_Transaction_Intensity', 'State_Price_Tier', 'Township_Uniqueness_Index',
    'Price_to_PSF_Ratio', 'Tenure_Type_Interaction', 'Is_Capital_City', 
    'Transaction_Volume_Flag'
]
print(df_fe[new_features].head())

Original dataset shape: (2000, 15)

Original columns: ['Township', 'Area', 'State', 'Tenure', 'Type', 'Median_Price', 'Median_PSF', 'Transactions', 'Price_per_Transaction', 'HighDemand', 'PriceTier', 'Freehold_Flag', 'Is_Central', 'Log_Median_Price', 'Log_Median_PSF']

1. Creating Area_Transaction_Intensity...
2. Creating State_Price_Tier...
3. Creating Township_Uniqueness_Index...
4. Creating Price_to_PSF_Ratio...
5. Creating Tenure_Type_Interaction...
6. Creating Is_Capital_City...
7. Creating Transaction_Volume_Flag...

FEATURE ENGINEERING COMPLETED
Original dataset shape: (2000, 15)
New dataset shape: (2000, 22)

SAVING AS CSV FILE
Feature-engineered dataset saved as: malaysia_house_prices_feature_engineered.csv
File saved successfully!
Rows: 2000, Columns: 22
Location: malaysia_house_prices_feature_engineered.csv

All columns in the saved CSV file:
 1. Township
 2. Area
 3. State
 4. Tenure
 5. Type
 6. Median_Price
 7. Median_PSF
 8. Transactions
 9. Price_per_Transaction
10. Hig